In [1]:
import pandas as pd
import seaborn as sns

In [2]:
data = pd.read_csv("clean_dataset.csv")
data.head()

C:\Users\saeed\AppData\Local\Temp\ipykernel_16064\1596611376.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("clean_dataset.csv")


,year_of_event,event_dates,event_name,event_distance/length(km),event_number_of_finishers,athlete_performance,athlete_club,athlete_country,athlete_year_of_birth,athlete_gender,athlete_age,athlete_average_speed,athlete_id
0,2018,06.01.2018,6 Stunden Self-Transcendence Sri Chinmoy Smole...,60.375,9,6.0,Мкшч,RUS,1966.0,M,50,10.063,22
1,2018,06.01.2018,6 Stunden Self-Transcendence Sri Chinmoy Smole...,59.750,9,6.0,Лично,RUS,1990.0,M,23,9.958,23
2,2018,06.01.2018,6 Stunden Self-Transcendence Sri Chinmoy Smole...,56.250,9,6.0,Ахиллес,RUS,1995.0,M,23,9.375,24
3,2018,06.01.2018,6 Stunden Self-Transcendence Sri Chinmoy Smole...,56.000,9,6.0,Аматар,RUS,1956.0,M,60,9.333,25
4,2018,06.01.2018,6 Stunden Self-Transcendence Sri Chinmoy Smole...,51.250,9,6.0,Endurance,RUS,1986.0,M,23,8.542,27


In [4]:
data.isna().any()

year_of_event                False
event_dates                  False
event_name                   False
event_distance/length(km)     True
event_number_of_finishers    False
athlete_performance          False
athlete_club                 False
athlete_country              False
athlete_year_of_birth        False
athlete_gender               False
athlete_age                  False
athlete_average_speed         True
athlete_id                   False
dtype: bool